Importing Libraries

In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 

Uploading Dataset

In [ ]:
from google.colab import files
files.upload()

Saving posts.csv to posts.csv
Saving users.csv to users.csv
Saving views.csv to views.csv


{'posts.csv': b'_id,title,category, post_type\n5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog\n5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Information Technology,blog\n5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog\n5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork\n5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog\n5d80e7c16c53455f896e6014,Raghavan Committee - 2002.,Competition Laws,blog\n5d80ecfd6c53455f896e601a,Let\'s discuss some Case laws!,Competition Laws,blog\n5d81323a6c53455f896e6044,Forms of Cartel.,Competition Laws,blog\n5d9b3514979d5962253c2f90,Biodiversity,Eco System,blog\n5d9b950768671220a1b2b153,Economic policy,Economic Policies,blog\n5da745b6019399436815c4cd,Corporate design,Graphic|Graphic Design,artwork\n5dada695610ba040fbfdf585,Trident,Drawings,artwork\n5dada734610ba040fbfdf588,Lord shiva,Drawings,artwork\n5dada798610ba040fbfdf58b,Smart City,Painting,artwork\n5da

Reading CSV files

In [ ]:
df1=pd.read_csv("posts.csv", encoding='unicode_escape', error_bad_lines=False)
df2=pd.read_csv("users.csv", encoding='unicode_escape', error_bad_lines=False)
df3=pd.read_csv("views.csv", encoding='unicode_escape', error_bad_lines=False)

Data pre Processing

In [ ]:
df1['post_no']=[x for x in range(0,df1.shape[0])]
df1.head(3)

,_id,title,category,post_type,post_no
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog,0
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,1
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog,2


In [ ]:
#checking for null values
df1.isnull().sum()

_id            0
title          0
category      28
 post_type     0
post_no        0
dtype: int64

In [ ]:
#replacing null valued catogries by any
df1.fillna("", inplace=True)

In [ ]:
#checking for duplicates
df1.drop_duplicates(inplace=True)
df1.shape

(493, 5)

In [ ]:
df2['user_no']=[x for x in range(0,df2.shape[0])]
df2.head(3)

,_id,name,gender,academics,user_no
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,0
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate,1
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate,2


In [ ]:
#checking for null values
df2.isnull().sum()

_id          0
name         0
gender       0
academics    0
user_no      0
dtype: int64

In [ ]:
#removing duplicate values if present
df2.drop_duplicates(keep=False, inplace=True)
df2.shape

(118, 5)

In [ ]:
df3.head(3)

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z


In [ ]:
#checking for null values
df3.isnull().sum()

user_id      0
post_id      0
timestamp    0
dtype: int64

In [ ]:
#removing duplicate values if any
df3.duplicated(['user_id','post_id'], keep=False)
df3.shape

(1449, 3)

In [ ]:
#droping timestamp cloumn
df3.drop(['timestamp'], axis=1, inplace=True)

sorting values based on post id 

In [ ]:

df3=df3.sort_values('post_id')
df3.reset_index(inplace=True)

Merging all the 3 dataframes

In [ ]:
df3=pd.merge(df1,df3,left_on='_id', right_on='post_id')
df3=pd.merge(df2,df3,left_on='_id', right_on='user_id')
df3.head(3)

,_id_x,name,gender,academics,user_no,_id_y,title,category,post_type,post_no,index,user_id,post_id
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog,0,809,5d60098a653a331687083238,5d62abaa65218653a132c956
1,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,0,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog,1,1223,5d60098a653a331687083238,5d6d39567fa40e1417a4931c
2,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate,0,5dbc631f99cbb90e4339c7fd,Calligraphy,Drawings|Calligraphy,artwork,20,1410,5d60098a653a331687083238,5dbc631f99cbb90e4339c7fd


Creating a new dataframe with all the columns that we are going to use for building a recomendation system

In [ ]:
title__=df3.groupby('title')['title'].agg('unique')
title__=list(title__)
title_=[]
for i in title__:
  Str = ' '.join([str(ele) for ele in i])
  title_.append(Str)

In [ ]:
gender__=df3.groupby('title')['gender'].agg('unique')
gender__=list(gender__)
gender_=[]
for i in gender__:
  Str = ' '.join([str(ele) for ele in i])
  gender_.append(Str)
  

In [ ]:
academics__=df3.groupby('title')['academics'].agg('unique')
academics__=list(academics__)
academics_=[]
for i in academics__:
  Str = ' '.join([str(ele) for ele in i])
  academics_.append(Str)

In [ ]:
category__=df3.groupby('title')['category'].agg('unique')
category__=list(category__)
category_=[]
for i in category__:
  Str = ' '.join([str(ele) for ele in i])
  category_.append(Str)

In [ ]:
post_type__=df3.groupby('title')[' post_type'].agg('unique')
post_type__=list(post_type__)
post_type_=[]
for i in post_type__:
  Str = ' '.join([str(ele) for ele in i])
  post_type_.append(Str)

In [ ]:
df = {'Title':title_,'Post_Type':post_type_, 'category':category_, 'ReadBy_gender': gender_, 'academics':academics_}

In [ ]:
df= pd.DataFrame(df)

In [ ]:
df.head(3)

,Title,Post_Type,category,ReadBy_gender,academics
0,'The Virtual ME',artwork,Fashion Design|Visual Arts|Graphic Design|Arti...,male female,graduate undergraduate
1,10 Best Artificial Intelligence (AI) Courses O...,blog,Computer Technology|Machine Learning,male,undergraduate
2,10 Best UI UX Design Course & Certification [2...,blog,Computer Technology|Design and Analysis of Alg...,male,undergraduate


In [ ]:
def add_space(Stri):
    new_str = Stri.replace('|',' ')
    return new_str

In [ ]:
df['category'] = df['category'].apply(add_space)
df["post_no"]=[x for x in range(0,df.shape[0])]
df.head(3)

,Title,Post_Type,category,ReadBy_gender,academics,post_no
0,'The Virtual ME',artwork,Fashion Design Visual Arts Graphic Design Arti...,male female,graduate undergraduate,0
1,10 Best Artificial Intelligence (AI) Courses O...,blog,Computer Technology Machine Learning,male,undergraduate,1
2,10 Best UI UX Design Course & Certification [2...,blog,Computer Technology Design and Analysis of Alg...,male,undergraduate,2


#Building Content Based Recomendation system

In [ ]:
def combine_data(data):
  features=[]
  for i in range(0,df.shape[0]):
    features.append(df['Title'][i]+' '+ df['Post_Type'][i]+' ' + df['category'][i]+' ' + df['ReadBy_gender'][i]+ ' ' + df['academics'][i])
  return features

In [ ]:
combined_data=combine_data(df)

In [ ]:
cv=CountVectorizer()
matrix=cv.fit_transform(combined_data)


In [ ]:
cs=cosine_similarity(matrix)
print(cs)

[[1.         0.09805807 0.30769231 ... 0.24019223 0.23652496 0.13074409]
 [0.09805807 1.         0.49029034 ... 0.20412415 0.22613351 0.25      ]
 [0.30769231 0.49029034 1.         ... 0.16012815 0.17739372 0.19611614]
 ...
 [0.24019223 0.20412415 0.16012815 ... 1.         0.24618298 0.27216553]
 [0.23652496 0.22613351 0.17739372 ... 0.24618298 1.         0.30151134]
 [0.13074409 0.25       0.19611614 ... 0.27216553 0.30151134 1.        ]]


In [ ]:
def get_post_no_from_post_id(index):
    return get_post_no_from_title(df1[df1['_id'] == index]['title'].values[0])

In [ ]:
def get_post_no_from_title(title):
    return df[df.Title == title]['post_no'].values[0]

In [ ]:
def get_title_from_post_no(post_no):
  return df[df.post_no == post_no]['Title'].values[0]

for now i am considering a custom input to see the results. 
We can use both used_id and post title to get similar posts.

In [ ]:
user_likes='5d62abaa65218653a132c956'
index=get_post_no_from_post_id(user_likes)

In [ ]:

user_likes='Ml and AI'
index= get_post_no_from_title(user_likes)

In [ ]:
similar_post = list(enumerate(cs[index]))

In [ ]:
sorted_similar_post = sorted(similar_post,key=lambda x:x[1],reverse=True)[1:]

#Result

In [ ]:
i=0
print('5 similar post to ' +user_likes+ ' are:\n')
for element in sorted_similar_post:
    print(get_title_from_post_no(element[0]))
    i=i+1
    if i>5:
        break

5 similar post to Ml and AI are:

10 Best Artificial Intelligence (AI) Courses Online for 2020
8 Best Machine Learning Courses for 2020
Learning...
Artificial Intelligence
Recommend Systems Machine Learning
Ethernet


#Colaborative Filtering

In [ ]:
df3["seen"]=1

In [ ]:
mat = df3.pivot_table(index='user_id', columns='post_id', values='seen')
mat = mat.fillna(0)
mat.head()

post_id,5d62abaa65218653a132c956,5d6d39567fa40e1417a4931c,5d7d23315720533e15c3b1ee,5d7d405e5720533e15c3b1f3,5d80dfbc6c53455f896e600e,5d80e7c16c53455f896e6014,5d80ecfd6c53455f896e601a,5d81323a6c53455f896e6044,5d9b3514979d5962253c2f90,5d9b950768671220a1b2b153,5da745b6019399436815c4cd,5dada695610ba040fbfdf585,5dada734610ba040fbfdf588,5dada798610ba040fbfdf58b,5dada82f610ba040fbfdf58e,5dada89b610ba040fbfdf591,5dadf6ae610ba040fbfdf5c8,5db2a7d099cbb90e4339c734,5db2b91a99cbb90e4339c749,5dbc622a99cbb90e4339c7f6,5dbc631f99cbb90e4339c7fd,5dc065ca24b883670268772f,5dd1751db802e41ed198b680,5dde6a91369b28584ecca156,5ddeb6e80eb5e25a8a07f065,5de179d80eb5e25a8a07f079,5de7971b8eab6401affbb137,5de8d73249e8203ff9219a74,5dea816a42a8854bf6eaba89,5dee9b5042a8854bf6eabaaf,5df64ad1cc709107827fb3dd,5e04fb252a37d20505da2a2b,5e1029f22a37d20505da2a79,5e27002d2a37d20505da2c13,5e281cf0025ae20886de2050,5e2867f6025ae20886de205a,5e294fd1c85ab714a7da6643,5e2d36ebc85ab714a7da66d1,5e2d3fbfc85ab714a7da66d5,5e2d447dc85ab714a7da66d7,...,5ecf23b87023451e66223944,5ecf818376027d35905cbf03,5ecf96e876027d35905cbf46,5ecfa0ca76027d35905cbf57,5ecfafcc76027d35905cbfb0,5ecfb35c76027d35905cbfbc,5ecfffb976027d35905cc0e9,5ed0007a76027d35905cc0ea,5ed09ad076027d35905cc15f,5ed0e20776027d35905cc2fe,5ed0e31a76027d35905cc302,5ed0e65976027d35905cc356,5ed0f0c576027d35905cc3c7,5ed13d2876027d35905cc4c2,5ed141aa76027d35905cc4c9,5ed175f176027d35905cc554,5ed175f476027d35905cc562,5ed1ff0276027d35905cc60d,5ed229d976027d35905cc666,5ed2378276027d35905cc6b5,5ed237d676027d35905cc6bd,5ed2383776027d35905cc6ca,5ed238d376027d35905cc6e5,5ed23a6376027d35905cc73b,5ed23b0776027d35905cc746,5ed23b6276027d35905cc74f,5ed23bb876027d35905cc759,5ed23c1876027d35905cc761,5ed23c8676027d35905cc780,5ed23cb876027d35905cc788,5ed23cf876027d35905cc790,5ed23d4276027d35905cc798,5ed23e4d76027d35905cc7b8,5ed2502b76027d35905cc7db,5ed3476576027d35905cca1d,5ed3791976027d35905ccab6,5ed3820f76027d35905ccac8,5ed3ef4cbd514d602c1530f2,5ed415c6bd514d602c15312d,5ed4cbadbd514d602c1531a6
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5d610ae1653a331687083239,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d618359fc5fcf3bdd9a0910,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d6d2bb87fa40e1417a49315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7c994d5720533e15c3b1e9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import operator

In [ ]:
def similar_users(user_id, matrix, k=3):
    
    user = mat[mat.index == user_id]
    
    
    other_users = mat[mat.index != user_id]
    
    
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    
    indices = other_users.index.tolist()
    
    
    index_similarity = dict(zip(indices, similarities))
    
    
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users
    


In [ ]:
current_user = '5d60098a653a331687083238'

similar_user_indices = similar_users(current_user, mat)

In [ ]:
similar_post=[]
for i in similar_user_indices:
  for j in range(df3.shape[0]):
    if(df3["user_id"][j]==i):
      similar_post.append(df3["title"][j])

In [ ]:
def recommend_item(user_index, similar_user_indices, matrix, items=5):
    
    
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    
    similar_users = similar_users.mean(axis=0)
    
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    
    
    user_df = matrix[matrix.index == user_index]
    
    user_df_transposed = user_df.transpose()
    
    user_df_transposed.columns = ['rating']
    
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    
    post_unseen = user_df_transposed.index.tolist()
    
    
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(post_unseen)]
    
    similar_users_df_ordered = similar_users_df.sort_values(by=['mean'], ascending=False)
      
    top_n_post = similar_users_df_ordered.head(items)
    top_n_post_indices = top_n_post.index.tolist()
    
    post_information = df1[df1['_id'].isin(top_n_post_indices)]
    
    return post_information #items


In [ ]:
items=recommend_item('5d60098a653a331687083238', similar_user_indices, mat)

In [ ]:
items

,_id,title,category,post_type,post_no
425,5ec3c09274f7660d73aa1229,Dakrai Artwork.,Pen and ink,artwork,425
429,5ec57914f2781131cc7e51c8,The Power of Indian Audience.,Art,skill,429
430,5ec57acaf2781131cc7e51d5,Ruins.,Painting,artwork,430
448,5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork,448
463,5ed0e31a76027d35905cc302,Designing Cmos circuit from Boolean expression...,,project,463


#Result

In [ ]:
print('5 recomended post are:\n')
for element in items['title']:
    print(element)

5 recomended post are:

Dakrai Artwork.
The Power of Indian Audience.
Ruins.
Faith in yourself
Designing Cmos circuit from Boolean expressions (Microwind)
